### MỤC TIÊU
Notebook này thực hiện bước **Tiền xử lý Dữ liệu (Data Preprocessing)** nhằm chuyển đổi dữ liệu thô thành dạng vector toán học chuẩn, sẵn sàng cho việc huấn luyện mô hình với các nhiệm vụ chính:

1.  **Làm sạch dữ liệu (Cleaning):**
    * Loại bỏ các đặc trưng định danh vô nghĩa (`CLIENTNUM`).
    * Loại bỏ các biến gây rò rỉ dữ liệu (Data Leakage) như `Naive_Bayes_...` để đảm bảo tính trung thực của mô hình.

2.  **Phân chia dữ liệu (Splitting):**
    * Tách tập dữ liệu thành hai phần **Huấn luyện (Train)** và **Kiểm thử (Test)** (tỷ lệ 80/20) thủ công bằng NumPy.
    * Mục đích: Đảm bảo đánh giá khách quan hiệu năng dự báo trên tập dữ liệu chưa từng gặp.

3.  **Mã hóa đặc trưng (Encoding):**
    * **Ordinal Encoding:** Áp dụng cho biến có thứ bậc (`Education_Level`, `Income_Category`, `Card_Category`) để giữ lại thông tin về mức độ.
    * **One-Hot Encoding:** Áp dụng cho biến danh nghĩa (`Gender`, `Marital_Status`) để tránh tạo ra thứ tự giả.

4.  **Chuẩn hóa dữ liệu (Scaling):**
    * Sử dụng phương pháp **Robust Scaling** (dựa trên Median và IQR) cho các biến số liên tục.
    * Mục đích: Giảm ảnh hưởng của Outliers và giúp thuật toán Gradient Descent hội tụ nhanh hơn.

5.  **Tạo đặc trưng mới (Feature Engineering):**
    * Xây dựng biến phái sinh `Avg_Trans_Amt` (Giá trị trung bình mỗi giao dịch) từ `Total_Trans_Amt` và `Total_Trans_Ct`.

6.  **Lưu trữ (Persisting):**
    * Xuất dữ liệu sau xử lý dưới dạng cấu trúc mảng **NumPy (.npy)** (Train/Test riêng biệt).
    * Mục đích: Tối ưu hóa tốc độ đọc/ghi và đảm bảo tính nhất quán dữ liệu cho bước Modeling.
___

In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath(os.path.join('..')))
from src import data_processing as dp

import importlib
importlib.reload(dp) 

print("Sẵn sàng xử lý dữ liệu với NumPy!")

Sẵn sàng xử lý dữ liệu với NumPy!


In [2]:
raw_path = '../data/raw/BankChurners.csv'
# Lấy ma trận 2D từ file CSV
header, data = dp.load_data_for_preprocessing(raw_path)
# Loại bỏ các cột không cần thiết
header_clean, data_clean = dp.clean_data_numpy(header, data)


--- Đang tải dữ liệu 2D từ ../data/raw/BankChurners.csv ---
Kích thước dữ liệu (Rows, Cols): (10127, 23)
Đã loại bỏ cột nhiễu. Kích thước mới: (10127, 19)


In [3]:
# Xác định cột Target
target_col = 'Attrition_Flag'
idx_target = np.where(header_clean == target_col)[0][0]

# Tách y: Mã hóa Attrited -> 1, Existing -> 0
y_raw = data_clean[:, idx_target]
y = np.where(y_raw == 'Attrited Customer', 1, 0)

# Tách X: Lấy tất cả cột trừ Target
mask_X = np.arange(len(header_clean)) != idx_target
X = data_clean[:, mask_X]
X_header = header_clean[mask_X]

# Gọi hàm từ data_processing.py để tính thêm cột Avg_Trans_Amt
print(f"Số cột trước khi Feature Engineering: {X.shape[1]}")
X, X_header = dp.feature_engineering_numpy(X, X_header)
print(f"Số cột sau khi Feature Engineering: {X.shape[1]}")

Số cột trước khi Feature Engineering: 18
--- Feature Engineering: Tạo cột Avg_Trans_Amt ---
-> Đã thêm 1 đặc trưng mới. Shape: (10127, 19)
Số cột sau khi Feature Engineering: 19


In [4]:
# Chia dữ liệu trước khi tiền xử lý để tránh rò rỉ dữ liệu
X_train_raw, X_test_raw, y_train, y_test = dp.train_test_split_numpy(X, y, test_size=0.2)

print(f"Train set: {X_train_raw.shape}")
print(f"Test set:  {X_test_raw.shape}")

-> Chia dữ liệu: Train (8102), Test (2025)
Train set: (8102, 19)
Test set:  (2025, 19)


In [5]:
# Khởi tạo Preprocessor
preprocessor = dp.NumpyPreprocessor()

# Fit (Học) trên tập TRAIN
preprocessor.fit(X_header, X_train_raw)

# Transform (Áp dụng) lên cả TRAIN và TEST
X_train_processed, new_header = preprocessor.transform(X_train_raw)
X_test_processed, _ = preprocessor.transform(X_test_raw)

print("--- Kết quả Preprocessing ---")
print(f"Số lượng Features ban đầu: {len(X_header)}")
print(f"Số lượng Features sau khi One-Hot: {len(new_header)}")
print(f"Mẫu tên features mới: {new_header[:5]}")

--- Đang 'Fit' trên tập Train ---
--- Kết quả Preprocessing ---
Số lượng Features ban đầu: 19
Số lượng Features sau khi One-Hot: 23
Mẫu tên features mới: ['Customer_Age' 'Gender_F' 'Gender_M' 'Dependent_count' 'Education_Level']


In [6]:
output_dir = '../data/processed/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lưu tập Train
dp.save_numpy(output_dir + 'train.npy', X_train_processed, y_train, new_header)

# Lưu tập Test
dp.save_numpy(output_dir + 'test.npy', X_test_processed, y_test, new_header)

Đã lưu file: ../data/processed/train.npy
Đã lưu file: ../data/processed/test.npy
